## Script to calculate the long term and shorter term runoff ratio from the VIC 

In [1]:
import numpy as np
import pandas as pd
import ipyparallel as pr
import livneh.tools as lt
import rhessys.utilities as rt
import time

In [2]:
c = pr.Client() # start the parallel environment

In [3]:
view = c.load_balanced_view() # access the first 4 engines

In [4]:
%%px
import pandas as pd
import numpy as np
import rhessys.utilities as rt
dates = pd.read_pickle('/Volumes/data/projects/Budyko_vic/timecode.pcl')
fluxes_columns = ['y','m','d','ET','R','BF','sm1','sm2','sm3','SWE','Cs','Qs','Ql','Qg','NR','PEText','PETtrc','PETsrc']

In [5]:
def fluxproc(idx,fl):
    flux = pd.read_table(fl, sep='\t', names = fluxes_columns) # read fluxes
    flux.index = pd.DatetimeIndex(dates)
    
    
    return idx,np.nanmean(flux.R.as_matrix())

In [6]:
files = pd.read_pickle('/Volumes/data/projects/Budyko_vic/forcing_fluxes_filenames_lat_lon_index.df')

In [7]:
res = view.map(fluxproc,files.indexer,files.flux)

In [9]:
while res.ready() == False:
    perc = round(float(res.progress)/float(len(files))*100.,2)
    print '#'*int(perc/4) # print a simple progress bar
    print str(perc)+'% Completed'
    time.sleep(60) # wait 60 seconds

#
5.94% Completed
#
6.4% Completed
#
6.85% Completed
#
7.31% Completed
#
7.76% Completed
##
8.22% Completed
##
8.67% Completed
##
9.12% Completed
##
9.55% Completed
##
10.0% Completed
##
10.44% Completed
##
10.87% Completed
##
11.31% Completed
##
11.75% Completed
###
12.19% Completed
###
12.63% Completed
###
13.07% Completed
###
13.51% Completed
###
13.95% Completed
###
14.39% Completed
###
14.83% Completed
###
15.26% Completed
###
15.71% Completed
####
16.15% Completed
####
16.59% Completed
####
17.03% Completed
####
17.47% Completed
####
17.91% Completed
####
18.35% Completed
####
18.79% Completed
####
19.23% Completed
####
19.66% Completed
#####
20.1% Completed
#####
20.54% Completed
#####
20.97% Completed
#####
21.41% Completed
#####
21.85% Completed
#####
22.27% Completed
#####
22.71% Completed
#####
23.14% Completed
#####
23.58% Completed
######
24.02% Completed
######
24.45% Completed
######
24.88% Completed
######
25.32% Completed
######
25.75% Completed
######
26.18% Completed

In [10]:
idx,R = zip(*res.result)

In [12]:
np.savez_compressed('./data/mean_runoff.npz',meanrunoff=R,idx=idx) # save the data